# Analysis: How do annual members and casual riders use Cyclistic bikes differently?

As the main goal is to determine a trend for each member type we will focus on opposite them all along this exploratory data analysis.

## Proportion

First, let us see the proportion of annual members & casual riders over these past 12 months.

In [1]:
SELECT
    member_casual,
    CAST(
        CAST (COUNT (*) AS FLOAT)* 100 / 
            (
            SELECT CAST (COUNT (*) AS FLOAT) 
            FROM [Cyclistic].[dbo].[twelve_months_trip]
            )
            AS DECIMAL(5,2) 
        ) AS proportion
FROM [Cyclistic].[dbo].[twelve_months_trip]
GROUP BY member_casual

(2 rows affected)

Total execution time: 00:00:04.516

member_casual,proportion
member,58.75
casual,41.25


As we can see, there are 41,25% of casual members. It explains why Cyclistic company is interested in designing a new marketing strategy to convert casual riders into annual members.

## Average trip duration

### Hebdomadary average trip duration for each member.

In [2]:
SELECT
    member_casual,
    AVG ((DATEDIFF(minute, [started_at], [ended_at]))) AS average_trip_duration_in_minutes
FROM [Cyclistic].[dbo].[twelve_months_trip]
GROUP BY member_casual

(2 rows affected)

Total execution time: 00:00:03.914

member_casual,average_trip_duration_in_minutes
member,12
casual,25


First insight, we can see that the casual riders use the bikes twice longer than the annual members. Now let us go see the average trip duration for each weekday.

### Daily average trip duration

In [10]:
SELECT
    DATENAME(weekday, started_at) AS day_of_week,
    AVG ((DATEDIFF(minute, [started_at], [ended_at]))) AS average_trip_duration_in_minutes
FROM [Cyclistic].[dbo].[twelve_months_trip]
GROUP BY DATENAME(weekday, started_at)
ORDER BY
    CASE DATENAME(weekday, started_at)
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(7 rows affected)

Total execution time: 00:00:04.612

day_of_week,average_trip_duration_in_minutes
lundi,17
mardi,15
mercredi,15
jeudi,16
vendredi,17
samedi,21
dimanche,22


Here we can see that the bike-share service is more used during the weekend. Next, let us see which type of user uses the bikes the longest each day.

## Member type with the longest average trip duration (each day of the week)

### Member vs Casual

In [27]:
WITH daily_member_average_trip_duration AS 
(
    SELECT
        DATENAME(weekday, started_at) AS day_of_week,
        AVG ((DATEDIFF(minute, [started_at], [ended_at]))) AS member_daily_average_trip_duration_in_minutes
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'member'
    GROUP BY 
        DATENAME(weekday, started_at)
)
SELECT
    daily_casual_average_trip_duration.*,
    daily_member_average_trip_duration.member_daily_average_trip_duration_in_minutes
FROM
(
    SELECT
        DATENAME(weekday, started_at) AS day_of_week,
        AVG ((DATEDIFF(minute, [started_at], [ended_at]))) AS casual_daily_average_trip_duration_in_minutes
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'casual'
    GROUP BY 
        DATENAME(weekday, started_at)
) AS daily_casual_average_trip_duration
FULL OUTER JOIN daily_member_average_trip_duration
ON daily_casual_average_trip_duration.day_of_week = daily_member_average_trip_duration.day_of_week
ORDER BY
    CASE daily_casual_average_trip_duration.day_of_week
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(7 rows affected)

Total execution time: 00:00:06.652

day_of_week,casual_daily_average_trip_duration_in_minutes,member_daily_average_trip_duration_in_minutes
lundi,26,12
mardi,22,11
mercredi,22,12
jeudi,22,12
vendredi,24,12
samedi,28,14
dimanche,29,14


Obvious results here, as the average trip duration is higher for casual riders than for annual members. Let us sum it up in another table.

### Rider with the highest daily average trip duration

In [28]:
WITH trip_duration_per_day_per_membertype AS 
(
        SELECT
            DATENAME(weekday, started_at) AS day_of_week,
            member_casual,
            AVG ((DATEDIFF(minute, [started_at], [ended_at]))) AS average_trip_duration_in_minutes
        FROM [Cyclistic].[dbo].[twelve_months_trip]
        GROUP BY
            DATENAME(weekday, started_at),
            member_casual
),
trip_duration_per_days_per_membertype_ranked AS
(
SELECT
    ROW_NUMBER() OVER (PARTITION BY day_of_week ORDER BY average_trip_duration_in_minutes DESC) AS trip_duration_ranked,
    *
    FROM
        trip_duration_per_day_per_membertype
)
SELECT
    day_of_week,
    member_casual
FROM
    trip_duration_per_days_per_membertype_ranked
WHERE
    trip_duration_ranked = 1
ORDER BY
    CASE day_of_week
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(7 rows affected)

Total execution time: 00:00:05.800

day_of_week,member_casual
lundi,casual
mardi,casual
mercredi,casual
jeudi,casual
vendredi,casual
samedi,casual
dimanche,casual


We studied the average trip duration in different ways, now let us see the difference between casual riders and annual members through the daily number of uses.

## Which user uses more the bike-share service daily?

### First, let us see it through numbers

In [32]:
WITH number_of_member_rides AS 
(
    SELECT
        DATENAME(weekday, started_at) AS day_of_week,
        COUNT(*) AS member_rides
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'member'
    GROUP BY 
        DATENAME(weekday, started_at)
)
SELECT
    number_of_casual_rides.*,
    number_of_member_rides.member_rides
FROM
(
    SELECT
        DATENAME(weekday, started_at) AS day_of_week,
        COUNT(*) AS casual_rides
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'casual'
    GROUP BY 
        DATENAME(weekday, started_at)
) AS number_of_casual_rides
FULL OUTER JOIN number_of_member_rides
ON number_of_casual_rides.day_of_week = number_of_member_rides.day_of_week
ORDER BY
    CASE number_of_casual_rides.day_of_week
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(7 rows affected)

Total execution time: 00:00:05.520

day_of_week,casual_rides,member_rides
lundi,219400,372804
mardi,203922,422648
mercredi,215291,430355
jeudi,229294,408883
vendredi,256138,361909
samedi,398425,347538
dimanche,341402,310301


As we can see, during weekdays, annual members ride more than casual riders. But during the weekend, casual riders use the bike-share service more. Now let us see those results in a summary table.

### Casual vs Member: Aggregated

In [33]:
WITH rides_per_day_per_membertype AS 
(
        SELECT
            DATENAME(weekday, started_at) AS day_of_week,
            member_casual,
            COUNT(*) AS rides
        FROM [Cyclistic].[dbo].[twelve_months_trip]
        GROUP BY
            DATENAME(weekday, started_at),
            member_casual
),
ride_per_days_per_membertype_ranked AS
(
SELECT
    ROW_NUMBER() OVER (PARTITION BY day_of_week ORDER BY rides DESC) AS rides_ranked,
    *
    FROM
        rides_per_day_per_membertype
)
SELECT
    day_of_week,
    member_casual,
    rides
FROM
    ride_per_days_per_membertype_ranked
WHERE
    rides_ranked = 1
ORDER BY
    CASE day_of_week
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(7 rows affected)

Total execution time: 00:00:04.674

day_of_week,member_casual,rides
lundi,member,372804
mardi,member,422648
mercredi,member,430355
jeudi,member,408883
vendredi,member,361909
samedi,casual,398425
dimanche,casual,341402


We have seen that casual riders tend to use more the bike-share service during the weekend than the annual members. Now let us see what type of bikes each member type prefers. The following bikes are available:

\- Classic bikes

\- Electric bikes

\- Docked bikes

## Bike prefered by casual riders/annual members

In [44]:
WITH rideable_type_member AS 
(
    SELECT
        rideable_type,
        COUNT(member_casual) AS uses_member
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'member'
    GROUP BY rideable_type
)
SELECT
    rideable_type_casual.*,
    rideable_type_member.uses_member
FROM
(
    SELECT
        rideable_type,
        COUNT(member_casual) AS uses_casual
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    WHERE member_casual = 'casual'
    GROUP BY rideable_type
) AS rideable_type_casual
FULL OUTER JOIN rideable_type_member
ON rideable_type_casual.rideable_type = rideable_type_member.rideable_type
ORDER BY
    CASE rideable_type_casual.rideable_type
        WHEN 'lundi' THEN 1
        WHEN 'mardi' THEN 2
        WHEN 'mercredi' THEN 3
        WHEN 'jeudi' THEN 4
        WHEN 'vendredi' THEN 5
        WHEN 'samedi' THEN 6
        WHEN 'dimanche' THEN 7
    END

(3 rows affected)

Total execution time: 00:00:05.990

rideable_type,uses_casual,uses_member
electric_bike,638295,808848
classic_bike,1020729,1845590
docked_bike,204848,NULL


In a more aggregated way:

In [6]:
WITH rideable_type_ranked AS (
    SELECT
        member_casual,
        rideable_type,
        COUNT(member_casual) AS uses
    FROM [Cyclistic].[dbo].[twelve_months_trip]
    GROUP BY member_casual,
            rideable_type
),
rideable_type_for_members_ranked AS
(
    SELECT
        ROW_NUMBER() OVER (PARTITION BY member_casual ORDER BY uses DESC) AS uses_ranked,
        *
    FROM
        rideable_type_ranked
)
SELECT
    member_casual,
    rideable_type,
    uses
FROM rideable_type_for_members_ranked
WHERE uses_ranked = 1
ORDER BY uses DESC

(2 rows affected)

Total execution time: 00:00:04.424

member_casual,rideable_type,uses
member,classic_bike,1845590
casual,classic_bike,1020729


As we can see both casual riders and annual members prefer to use the classic bikes, so there is nothing to dig in. Let us take a look at how many/which stations are used for each member type

## Stations used

## <span style="font-size: 14px;">Let us start with the percentage of stations used for each member type.</span>

In [45]:
SELECT
    member_casual,
    COUNT (DISTINCT start_station_name) * 100 / (
        SELECT COUNT (DISTINCT start_station_name) AS stations
        FROM [Cyclistic].[dbo].[twelve_months_trip]
     ) AS percentage_stations_used
FROM [Cyclistic].[dbo].[twelve_months_trip]
GROUP BY member_casual

(2 rows affected)

Total execution time: 00:00:25.937

member_casual,percentage_stations_used
member,90
casual,95


As we can see, casual riders tend to use more stations than annual members. Now we will see which are the most used stations for each member type

### Top 10 Stations : Casual Riders

In [39]:
SELECT TOP 10
    start_station_name,
    COUNT (TRIM(start_station_name)) AS uses,
    CAST(start_lng AS FLOAT) AS started_lng,
    CAST(start_lat AS FLOAT) AS start_lat,
    CAST(end_lng AS FLOAT) AS end_lng,
    CAST(end_lat AS FLOAT) AS end_lat
FROM [Cyclistic].[dbo].[twelve_months_trip]
WHERE member_casual = 'casual'
GROUP BY start_station_name,
        start_lng,
        start_lat,
        end_lng,
        end_lat
ORDER BY uses DESC

(10 rows affected)

Total execution time: 00:02:35.058

start_station_name,uses,started_lng,start_lat,end_lng,end_lat
Streeter Dr & Grand Ave,11204,"-87,61","41,89","-87,61","41,89"
DuSable Lake Shore Dr & Monroe St,8479,"-87,62","41,88","-87,62","41,88"
Streeter Dr & Grand Ave,8264,"-87,61","41,89","-87,62","41,88"
DuSable Lake Shore Dr & Monroe St,6844,"-87,62","41,88","-87,61","41,89"
Millennium Park,6785,"-87,62","41,88","-87,62","41,88"
Michigan Ave & Oak St,5138,"-87,62","41,9","-87,62","41,9"
Shedd Aquarium,4521,"-87,62","41,87","-87,62","41,88"
Dusable Harbor,4305,"-87,61","41,89","-87,61","41,89"
Streeter Dr & Grand Ave,4114,"-87,61","41,89","-87,62","41,89"
Streeter Dr & Grand Ave,3801,"-87,61","41,89","-87,62","41,9"


### Top 10 Stations: Annual Members

In [40]:
SELECT TOP 10
    start_station_name,
    COUNT (TRIM(start_station_name)) AS uses,
    CAST(start_lng AS FLOAT) AS started_lng,
    CAST(start_lat AS FLOAT) AS start_lat,
    CAST(end_lng AS FLOAT) AS end_lng,
    CAST(end_lat AS FLOAT) AS end_lat
FROM [Cyclistic].[dbo].[twelve_months_trip]
WHERE member_casual = 'member'
GROUP BY start_station_name,
        start_lng,
        start_lat,
        end_lng,
        end_lat
ORDER BY uses DESC

(10 rows affected)

Total execution time: 00:02:26.486

start_station_name,uses,started_lng,start_lat,end_lng,end_lat
Ellis Ave & 60th St,13640,"-87,6","41,79","-87,6","41,79"
University Ave & 57th St,7920,"-87,6","41,79","-87,6","41,79"
Ellis Ave & 55th St,7137,"-87,6","41,79","-87,6","41,79"
Loomis St & Lexington St,5765,"-87,66","41,87","-87,65","41,87"
Kimbark Ave & 53rd St,4453,"-87,59","41,8","-87,6","41,79"
Kingsbury St & Kinzie St,3936,"-87,64","41,89","-87,64","41,88"
University Ave & 57th St,3648,"-87,6","41,79","-87,59","41,8"
Ellis Ave & 58th St,3510,"-87,6","41,79","-87,6","41,79"
Halsted St & Polk St,3478,"-87,65","41,87","-87,66","41,87"
Morgan St & Polk St,3387,"-87,65","41,87","-87,66","41,87"


Casual riders and annual members seem to not use the same stations (if we take a look at the 10 most used stations). We will see if can exploit those results.

## Conclusion : usefull insights

\- Casual riders tend to have trips with the highest average duration trip.

\- Casual riders use the bike-share service more during the weekend than annual members, but the other days, it is the contrary.

\- Casual riders use more stations than annual members.

\- Casual riders and annual members seem to not use the same stations (at least the top 10).